In [13]:
from utils import (
    get_cross_validation_results,
    get_final_model_performance,
    TransactionDataset,
    TuneHyperParams,
)

from constants import Columns, ModelConstants, ResamplingStrategy

In [14]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    FunctionTransformer,
    Normalizer,
    PowerTransformer,
    RobustScaler
)
from sklearn.compose import make_column_transformer, make_column_selector

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE, ADASYN


In [15]:
data = TransactionDataset().get_training_test_split()

In [16]:
log_transform = FunctionTransformer(
    func=np.log1p, inverse_func=np.expm1, check_inverse=False
)

order_mapping = [['0', '0.2', '0.4', '0.6', '0.8', '1']]

norm_and_one_hot_transforms = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY]),
    (Normalizer(), make_column_selector(dtype_include="number"))
)

log_and_ordinal_transforms = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY]),
    (log_transform, make_column_selector(dtype_include="number"))
)

power_transforms = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY]),
    (PowerTransformer(), make_column_selector(dtype_include="number"))
)

robust_transformation = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY]),
    (RobustScaler(), make_column_selector(dtype_include="number"))
)

In [17]:
knn_classifier = KNeighborsClassifier(
    n_jobs=ModelConstants.N_JOBS
)

In [18]:
smote_sampling = SMOTE(random_state=ModelConstants.RANDOM_STATE)
adasyn_sampling = ADASYN(random_state=ModelConstants.RANDOM_STATE)

In [19]:
column_transforms_only_model = Pipeline(
    [
        ("column_transformation", log_and_ordinal_transforms),
        ("sampling", smote_sampling),
        ("knn", knn_classifier)
    ]
)

In [20]:
tuning_params = {
    "column_transformation": [
        log_and_ordinal_transforms,
        norm_and_one_hot_transforms,
        power_transforms,
        robust_transformation,
        "passthrough"
    ],

    "sampling": [
        smote_sampling,
        adasyn_sampling,
    ],
    "sampling__sampling_strategy": [
        ResamplingStrategy.MINORITY_ONLY,
        ResamplingStrategy.ALL,
        0.2,
        0.3,
        0.4,
        0.5,
        0.6,
        0.7
    ],

    "knn__n_neighbors": [5, 7, 9, 11, 13, 15, 17, 19, 21, 25],
    "knn__weights": ["uniform", "distance"],
}

In [21]:
best_model = (
    TuneHyperParams()
    .random_grid_search(
     column_transforms_only_model,
     tuning_params
    )
    .fit_model(
        data.TRAINING.predictors,
        data.TRAINING.outcome
    )
    .get_best_model()
)

{'sampling__sampling_strategy': 0.3, 'sampling': ADASYN(random_state=123, sampling_strategy=0.3), 'knn__weights': 'distance', 'knn__n_neighbors': 15, 'column_transformation': ColumnTransformer(transformers=[('onehotencoder', OneHotEncoder(),
                                 ['Customer_Type']),
                                ('ordinalencoder',
                                 OrdinalEncoder(categories=[['0', '0.2', '0.4',
                                                             '0.6', '0.8',
                                                             '1']]),
                                 ['SpecificHoliday']),
                                ('robustscaler', RobustScaler(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x15caabf10>)])}
Best parameter (CV score: 0.625):


In [22]:
get_cross_validation_results(best_model, data.TRAINING.predictors, data.TRAINING.outcome)

,Metric for Training Set,Score
0,fit_timemean score,0.055133
1,score_timemean score,0.045694
2,5-fold CV F1 Scoremean score,0.625417
3,5-fold CV Accuracymean score,0.875564
4,5-fold CV Balanced Accuracymean score,0.789849
5,5-fold CV Matthew's Correlation Coefficientmea...,0.552930


In [23]:
get_final_model_performance(best_model, data.TRAINING, data.TESTING)

,Metric for Testing Set,Score
0,F1 Score,0.621656
1,Accuracy,0.878329
2,Balanced Accuracy,0.768873
3,Matthew's Correlation Coefficient,0.549505
